<a href="https://colab.research.google.com/github/LittleAri/ALICE/blob/main/ALICE_rCNN_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mask RCNN Training for ALICE

Train model ([Mask RCNN](https://github.com/matterport/Mask_RCNN)) on images of pinned insects.

#### Step 0
Mount Google Drive.

This step is useful if using Google Colab, as it's often easier to save documents there.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Step 1

Check that images have loaded properly.

This is a precautionary step which is helpful when using Google Colab, to check if images have been uploaded correctly.

In [ ]:
import os
import skimage.io as io

In [ ]:
errors = 0
non_errors = 0

for nm in os.listdir("balloon/val"):
  if "json" not in nm:
    try:
      image = io.imread("balloon/val/"+nm)
      non_errors = non_errors+1
    except:
      print(nm)
      errors = errors+1

print(["val",non_errors,errors])

['val', 15, 0]


#### Step 2

Import certain versions of Python packages.

In [ ]:
!pip install tensorflow==1.15.5
!pip install keras==2.1.0
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install numpy==1.21.5
!pip install tensorflow-gpu==1.15.5
!pip install scikit-image==0.16.2

     |████████████████████████████████| 110.5 MB 1.6 kB/s 
     |████████████████████████████████| 3.8 MB 30.1 MB/s 
     |████████████████████████████████| 20.1 MB 55.4 MB/s 
     |████████████████████████████████| 503 kB 46.9 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
     |████████████████████████████████| 2.9 MB 32.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=9d8bc273c6988a7f141b5e8a8c874f33062fce44916f00cd25487faed98f7594
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: ten

     |████████████████████████████████| 302 kB 5.2 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-3nxi5owm
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-3nxi5owm
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=111b795b52b6900f68392e7bee26e716ec37f16d5befeb996831c8e6f0a0fcda
  Stored in directory: /tmp/pip-ephem-wheel-cache-3s_8ls8s/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib
     |████████████████████████████████| 15.7 MB 5.5 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: pip's dependency resolver does not currently take i

     |████████████████████████████████| 411.0 MB 26 kB/s 
  Using cached numpy-1.18.5-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.16.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.18.5 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.5 which is incompatible.
jaxlib 0.3.2+cuda11.cudnn805 requires numpy>=1.19, but you have numpy 1.18.5 which is incompatible.
jax 0.3.4 requires numpy>=1.19, but you have numpy 1.18.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompa

     |████████████████████████████████| 26.5 MB 1.6 MB/s 
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:
      Successfully uninstalled scikit-image-0.18.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


#### Step 3:
Import libraries.

In [ ]:
import os
import sys
import json
import re
import random
import numpy as np
import matplotlib.pyplot as plt
import utils
import visualize
from visualize import display_images
import balloon
from model import log
import skimage
from google.colab import files as files_
import os
import tensorflow as tf
%matplotlib inline 

Using TensorFlow backend.


#### Step 4

Double check if there's a GPU.

In [ ]:
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


#### Step 5

Train model by building on top of pre-trained weights.

The path for the images / labels that will be used for trianing, should be specified in _folder_for_labels_. The path to the folder containing the model and the pre-trained weights should be specified in _folder_for_model_.

In [ ]:
folder_for_labels = balloon
folder_for_model = coco

!python3 balloon.py train --dataset=folder_for_labels --weights=folder_for_model

Using TensorFlow backend.
Weights:  coco
Dataset:  balloon
Logs:  /logs

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcn

#### Step 6

Download saved weights.

This step is helpful when using Google Colab. As weights are automatically saved, it is useful to automatically download them from the Colab directory. These are found in _../logs_.

Weights are automatically saved. If using Google Colab, it is useful to download these weights automatically.

In [ ]:
from google.colab import files

for fold in os.listdir("../logs"):
  # 1) Find all files:
  vals = []
  try:
    for file in os.listdir("../logs/"+fold):
      vals.append("../logs/"+fold+'/'+file)
  except:
    vals.append("../logs/"+fold)
  # 2) Filter files:
  final_vals = []
  for u in vals:
    if ("label" in u) and ("h5" in u):
      final_vals.append(u)
  # 3) Download weights:
  files.download(final_vals[-1]) 